In [1]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense

# Загружаем датасет
#df = pd.read_csv('/content/1400.xlsx - Structures-ADMET.csv')
# Разделяем на входные и выходные данные
# X = df.drop(['y1', 'y2'], axis=1)
# y = df['y2']
# len(y)


In [2]:
!pip install rdkit
from rdkit import Chem
from rdkit import DataStructs
from rdkit.Chem import AllChem, Draw, Descriptors
from rdkit.Chem.Draw import IPythonConsole
from sklearn.preprocessing import FunctionTransformer


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 49.2 MB/s eta 0:00:00


In [62]:
df = pd.read_csv('Big_data.csv')
len(df)

34909

In [63]:
df = df.drop(columns="Unnamed: 0")

In [64]:
# df = df.drop(columns=['molecule', 'Pictures', 'SMILES','Unnamed: 55', 'Unnamed: 56'], axis=1)
# df.dropna(inplace=True)
# df = df.replace(',', '.', regex=True)
# df =df.astype(float)
#df.dropna(inplace=True)
df = df.replace(',', '.', regex=True)
df = df.astype(float)
df = df.drop_duplicates(subset="0")
df = df[df['y'] < 40]
X = df.drop(columns=['y'], axis=1)
y = df['y']

In [65]:
X

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,1.021375,-2.071907,0.109130,2.028637,1.647268,-5.600501,-11.082684,-2.937554,8.876988,0.754120,...,-2.432685,10.301312,8.661780,2.696368,-7.467528,-4.141777,-6.063578,-5.465941,-12.236846,-0.982680
1,-0.473433,-2.432352,-3.370579,-2.080286,2.272056,-2.999003,-9.676762,-3.197955,5.274058,-1.527994,...,-0.093318,8.098873,5.970570,3.107769,-8.935454,-5.169111,-2.756884,-6.406341,-11.645088,-2.429996
2,0.496656,-3.803691,-2.876035,-2.808888,2.432933,-4.452638,-10.178493,-2.079107,3.339100,0.711161,...,0.402508,9.255146,5.377076,2.417149,-9.325192,-4.567752,-4.578742,-8.215970,-10.413951,-2.376374
3,-0.111485,-3.251434,-2.736400,-2.645135,2.578548,-5.707126,-8.472591,-2.248675,4.185829,-0.826409,...,1.283272,9.187645,5.323506,2.145598,-7.669068,-4.591479,-2.775401,-6.426649,-10.316141,-1.198285
4,2.230854,-2.023349,-6.237648,5.693612,0.467946,1.374880,-9.434242,-1.290353,2.973960,1.139288,...,-1.626027,9.080440,10.368519,-0.723060,-4.942924,-4.798901,-4.013011,-2.022188,-11.399353,1.204189
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34553,6.032118,-3.818947,-5.672975,10.221979,0.017723,-0.287975,-15.419767,-1.669053,6.423544,8.213655,...,-3.481098,11.742849,11.492210,-2.603210,-13.908297,-3.959302,-6.968355,-3.404389,-13.037302,-1.866105
34554,-1.979658,-19.632324,-18.466543,-4.907841,2.055253,-24.648535,-26.600960,3.486309,-7.942854,-5.433523,...,-2.430328,44.874100,-1.586750,-2.722094,-32.518353,-37.734820,-31.053936,-16.922432,-41.428772,8.600471
34555,-1.553332,0.581597,-2.938496,-2.241104,1.089104,-1.711056,-4.264384,-0.149229,0.072480,2.103765,...,-3.588788,2.150058,2.819722,2.405031,-3.316368,-2.733332,-0.609319,-2.404624,-5.406436,-1.135767
34556,0.987552,-6.262579,0.320636,3.445588,-0.409579,-0.047092,-10.905636,1.173008,6.466604,3.968681,...,-1.334474,13.137399,7.024012,-1.596717,-6.932823,-2.542508,-10.896585,-3.767301,-11.942197,-1.662916


In [24]:
len(df)

34353

In [ ]:
# X = df.drop(columns=['IC50, mmg/ml', 'CC50-MDCK, mmg/ml', 'SI'], axis=1)
# y = df['IC50, mmg/ml']


In [75]:
y

0         0.00
1         0.00
2         0.00
3         0.00
4         0.00
         ...  
34553    39.65
34554    39.68
34555    39.94
34556    39.96
34557    39.98
Name: y, Length: 34353, dtype: float64

In [35]:
def remove_outliers(col):
    mean = col.mean()
    std = col.std()
    threshold = 3 * std
    return col[abs(col - mean) < threshold]

X = X.apply(remove_outliers)

In [36]:
X

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,1.021375,-2.071907,0.109130,2.028637,1.647268,-5.600501,-11.082684,-2.937554,8.876988,0.754120,...,-2.432685,10.301312,8.661780,2.696368,-7.467528,-4.141777,-6.063578,-5.465941,-12.236846,-0.982680
1,-0.473433,-2.432352,-3.370579,-2.080286,2.272056,-2.999003,-9.676762,-3.197955,5.274058,-1.527994,...,-0.093318,8.098873,5.970570,3.107769,-8.935454,-5.169111,-2.756884,-6.406341,-11.645088,-2.429996
2,0.496656,-3.803691,-2.876035,NaN,2.432933,-4.452638,-10.178493,-2.079107,3.339100,0.711161,...,0.402508,9.255146,5.377076,2.417149,-9.325192,-4.567752,-4.578742,-8.215970,-10.413951,-2.376374
3,-0.111485,-3.251434,-2.736400,-2.645135,2.578548,-5.707126,-8.472591,-2.248675,4.185829,-0.826409,...,1.283272,9.187645,5.323506,2.145598,-7.669068,-4.591479,-2.775401,-6.426649,-10.316141,-1.198285
4,2.230854,-2.023349,-6.237648,5.693612,0.467946,1.374880,-9.434242,-1.290353,2.973960,1.139288,...,-1.626027,9.080440,10.368519,-0.723060,-4.942924,-4.798901,-4.013011,-2.022188,-11.399353,1.204189
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34553,6.032118,-3.818947,-5.672975,10.221979,0.017723,-0.287975,-15.419767,-1.669053,6.423544,8.213655,...,-3.481098,11.742849,11.492210,-2.603210,-13.908297,-3.959302,-6.968355,-3.404389,-13.037302,-1.866105
34554,-1.979658,NaN,NaN,NaN,2.055253,NaN,NaN,3.486309,NaN,NaN,...,-2.430328,NaN,NaN,-2.722094,NaN,NaN,NaN,NaN,NaN,NaN
34555,-1.553332,0.581597,-2.938496,-2.241104,1.089104,-1.711056,-4.264384,-0.149229,0.072480,2.103765,...,-3.588788,2.150058,2.819722,2.405031,-3.316368,-2.733332,-0.609319,-2.404624,-5.406436,-1.135767
34556,0.987552,-6.262579,0.320636,3.445588,-0.409579,-0.047092,-10.905636,1.173008,6.466604,3.968681,...,-1.334474,13.137399,7.024012,-1.596717,-6.932823,-2.542508,-10.896585,-3.767301,-11.942197,-1.662916


In [37]:
print(X.columns)
X['y'] = y
print(X.columns)

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '290', '291', '292', '293', '294', '295', '296', '297', '298', '299'],
      dtype='object', length=300)
Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '291', '292', '293', '294', '295', '296', '297', '298', '299', 'y'],
      dtype='object', length=301)


In [38]:
X['y']

0         0.00
1         0.00
2         0.00
3         0.00
4         0.00
         ...  
34553    39.65
34554    39.68
34555    39.94
34556    39.96
34557    39.98
Name: y, Length: 34353, dtype: float64

In [39]:
X.dropna(inplace=True)

In [66]:
X

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,1.021375,-2.071907,0.109130,2.028637,1.647268,-5.600501,-11.082684,-2.937554,8.876988,0.754120,...,-2.432685,10.301312,8.661780,2.696368,-7.467528,-4.141777,-6.063578,-5.465941,-12.236846,-0.982680
1,-0.473433,-2.432352,-3.370579,-2.080286,2.272056,-2.999003,-9.676762,-3.197955,5.274058,-1.527994,...,-0.093318,8.098873,5.970570,3.107769,-8.935454,-5.169111,-2.756884,-6.406341,-11.645088,-2.429996
2,0.496656,-3.803691,-2.876035,-2.808888,2.432933,-4.452638,-10.178493,-2.079107,3.339100,0.711161,...,0.402508,9.255146,5.377076,2.417149,-9.325192,-4.567752,-4.578742,-8.215970,-10.413951,-2.376374
3,-0.111485,-3.251434,-2.736400,-2.645135,2.578548,-5.707126,-8.472591,-2.248675,4.185829,-0.826409,...,1.283272,9.187645,5.323506,2.145598,-7.669068,-4.591479,-2.775401,-6.426649,-10.316141,-1.198285
4,2.230854,-2.023349,-6.237648,5.693612,0.467946,1.374880,-9.434242,-1.290353,2.973960,1.139288,...,-1.626027,9.080440,10.368519,-0.723060,-4.942924,-4.798901,-4.013011,-2.022188,-11.399353,1.204189
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34553,6.032118,-3.818947,-5.672975,10.221979,0.017723,-0.287975,-15.419767,-1.669053,6.423544,8.213655,...,-3.481098,11.742849,11.492210,-2.603210,-13.908297,-3.959302,-6.968355,-3.404389,-13.037302,-1.866105
34554,-1.979658,-19.632324,-18.466543,-4.907841,2.055253,-24.648535,-26.600960,3.486309,-7.942854,-5.433523,...,-2.430328,44.874100,-1.586750,-2.722094,-32.518353,-37.734820,-31.053936,-16.922432,-41.428772,8.600471
34555,-1.553332,0.581597,-2.938496,-2.241104,1.089104,-1.711056,-4.264384,-0.149229,0.072480,2.103765,...,-3.588788,2.150058,2.819722,2.405031,-3.316368,-2.733332,-0.609319,-2.404624,-5.406436,-1.135767
34556,0.987552,-6.262579,0.320636,3.445588,-0.409579,-0.047092,-10.905636,1.173008,6.466604,3.968681,...,-1.334474,13.137399,7.024012,-1.596717,-6.932823,-2.542508,-10.896585,-3.767301,-11.942197,-1.662916


In [41]:
y = X['y']
X = X.drop(columns=['y'], axis=1)

In [ ]:
# .astype(float)
# X = X.replace(',', '.', regex=True)
# X = X.astype(float)
# for i in X.columns:
#   for j in range(len(X[i])):
#     print(X[i][j])
#     X[i][j] = str(X[i][j]).replace(',', '.')
#   X[i] = X[i].astype(float)

array([0.        , 0.27662779, 0.58473143, 0.49142093, 0.02882619,
       0.71964815, 0.28650851, 0.56648854, 0.22984489, 0.37504866,
       0.13533156, 0.816611  , 0.74246127, 0.82133412, 0.71700792,
       0.31417286, 0.30664273, 0.60128955, 0.34731213, 0.49788425,
       0.36295244, 0.6747691 , 0.25874791, 0.52091163, 0.48304924,
       0.62233654, 0.96189605, 0.38888141, 0.69594672, 0.26309342,
       0.19349601, 0.1325371 , 0.61747761, 0.36640584, 0.54804532,
       0.14433639, 0.37669437, 0.3237569 , 0.59481606, 0.45521375,
       0.58194704, 0.69696606, 0.8114321 , 0.63686194, 0.35876174,
       0.65730108, 0.66506878, 0.66004814, 0.82111626, 0.68988028,
       0.27060444, 0.03220643, 0.54389986, 0.59898465, 0.85350932,
       0.29790266, 0.25871449, 0.56087009, 0.64037853, 0.5061372 ,
       0.538767  , 0.27632706, 0.8985477 , 0.21074885, 0.56355654,
       0.51984752, 0.18069077, 0.37401381, 0.42010843, 0.2144147 ,
       0.37734378, 0.40711545, 0.76409861, 0.43441677, 0.53337

In [42]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import seaborn as sb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from scipy.spatial import ConvexHull
from keras.optimizers import Adagrad

In [76]:
y = y.values.reshape((-1,1))

In [77]:
y

array([[ 0.  ],
       [ 0.  ],
       [ 0.  ],
       ...,
       [39.94],
       [39.96],
       [39.98]])

In [91]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=41)

In [92]:
from sklearn import preprocessing
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [93]:
y_train = scaler.fit_transform(y_train)
y_test = scaler.transform(y_test)

In [46]:
train_x

array([[0.48712355, 0.74446894, 0.46438734, ..., 0.63132451, 0.59900128,
        0.58399386],
       [0.55103146, 0.62700308, 0.43141859, ..., 0.531519  , 0.42593983,
        0.28943865],
       [0.34697311, 0.37277198, 0.61605301, ..., 0.50406156, 0.5336293 ,
        0.31286899],
       ...,
       [0.55727206, 0.38603005, 0.486511  , ..., 0.59346556, 0.41946169,
        0.34610658],
       [0.67870663, 0.28890524, 0.51020728, ..., 0.4027857 , 0.28562896,
        0.25069075],
       [0.23817628, 0.53956421, 0.40312737, ..., 0.35820472, 0.49714963,
        0.28980968]])

In [108]:
# Создаем модель нейронной сети
model = Sequential()
model.add(Dense(300, input_dim=X.shape[1], activation='relu'))
model.add(Dense(150, input_dim=300, activation='relu'))
model.add(Dense(1, activation='linear'))

# Создание оптимизатора Adagrad
optimizer = Adagrad(learning_rate=10)

# Компиляция модели с использованием оптимизатора Adagrad
model.compile(loss='MSE', optimizer='adam', metrics=['accuracy'])

# Обучаем модель
model.fit(x_train, y_train, epochs=200, batch_size=50)

# Предсказываем значения для новых данных
# new_data = pd.DataFrame({'feature_1': [1, 2, 3], 'feature_2': [4, 5, 6]})
# predictions = model.predict(test_x)

# for i in range(len(test_x)):
#   print(test_y[i], predictions[i])

Epoch 1/200
516/516 [==============================] - 3s 4ms/step - loss: 0.9464 - accuracy: 0.0000e+00
Epoch 2/200
516/516 [==============================] - 2s 4ms/step - loss: 0.8253 - accuracy: 0.0000e+00
Epoch 3/200
516/516 [==============================] - 2s 4ms/step - loss: 0.8097 - accuracy: 0.0000e+00
Epoch 4/200
516/516 [==============================] - 3s 6ms/step - loss: 0.7692 - accuracy: 0.0000e+00
Epoch 5/200
516/516 [==============================] - 3s 5ms/step - loss: 0.7631 - accuracy: 0.0000e+00
Epoch 6/200
516/516 [==============================] - 2s 4ms/step - loss: 0.7288 - accuracy: 0.0000e+00
Epoch 7/200
516/516 [==============================] - 2s 5ms/step - loss: 0.7075 - accuracy: 0.0000e+00
Epoch 8/200
516/516 [==============================] - 2s 5ms/step - loss: 0.6757 - accuracy: 0.0000e+00
Epoch 9/200
516/516 [==============================] - 3s 6ms/step - loss: 0.6678 - accuracy: 0.0000e+00
Epoch 10/200
516/516 [==============================] -

In [109]:
predictions = model.predict(x_test)
for i in range(len(x_test)):
  print(predictions[i], y_test[i])

Выходные данные были обрезаны до нескольких последних строк (5000).
[0.19562685] [-0.01837118]
[0.88191164] [1.69391007]
[0.8919196] [-0.36622044]
[-0.5235826] [1.01169408]
[-0.8324467] [-0.72755223]
[-0.5944487] [-0.76530331]
[0.7912508] [-0.8515915]
[-0.6269897] [-0.64935356]
[0.05025101] [0.15690171]
[0.7186111] [1.52133369]
[1.2012222] [9.66478149]
[-0.44229078] [-0.40397152]
[-0.6119758] [-0.7140697]
[-0.707392] [1.60762188]
[0.07187217] [-1.06461546]
[-0.43512726] [1.21662852]
[0.2885868] [5.84652915]
[-0.7252989] [-0.20443008]
[-0.7459091] [0.27824447]
[0.8444055] [-0.77339283]
[0.5573903] [0.4750894]
[-0.83637655] [-0.32846935]
[-1.0861038] [-1.10775956]
[-0.8975253] [-0.61160247]
[1.2590761] [0.93619191]
[0.45466417] [1.05483817]
[-0.02146453] [0.00859388]
[0.43836743] [0.35105013]
[-1.6777021] [-0.71137319]
[1.3257215] [0.84451071]
[-0.21461439] [0.75552602]
[0.19646281] [-0.58194091]
[-0.24953234] [0.67193433]
[-0.4871894] [-0.70058717]
[0.01350814] [-0.73294524]
[0.6201462]

In [103]:

from sklearn.metrics import mean_squared_error, r2_score


In [104]:

mse = mean_squared_error(y_test, predictions)


In [105]:
mse

1.0576620693510908

In [106]:

r2 = r2_score(y_test, predictions)


In [107]:
r2

-0.009167210484478883

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
!pip install rdkit

In [5]:
from rdkit import Chem, DataStructs
from rdkit.Chem import PandasTools, AllChem
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [6]:
df = pd.read_csv('/content/Big_data.csv')

In [12]:
df = df.replace('#REF!', ' ', regex=True)

In [13]:
df

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,y
0,1.021375,-2.071907,0.109130,2.028637,1.647268,-5.600501,-11.082684,-2.937554,8.876988,0.754120,...,10.301312,8.661780,2.696368,-7.467528,-4.141777,-6.063578,-5.465941,-12.236846,-0.982680,0.00
1,-0.473433,-2.432352,-3.370579,-2.080286,2.272056,-2.999003,-9.676762,-3.197955,5.274058,-1.527994,...,8.098873,5.970570,3.107769,-8.935454,-5.169111,-2.756884,-6.406341,-11.645088,-2.429996,0.00
2,0.496656,-3.803691,-2.876035,-2.808888,2.432933,-4.452638,-10.178493,-2.079107,3.339100,0.711161,...,9.255146,5.377076,2.417149,-9.325192,-4.567752,-4.578742,-8.215970,-10.413951,-2.376374,0.00
3,-0.111485,-3.251434,-2.736400,-2.645135,2.578548,-5.707126,-8.472591,-2.248675,4.185829,-0.826409,...,9.187645,5.323506,2.145598,-7.669068,-4.591479,-2.775401,-6.426649,-10.316141,-1.198285,0.00
4,2.230854,-2.023349,-6.237648,5.693612,0.467946,1.374880,-9.434242,-1.290353,2.973960,1.139288,...,9.080440,10.368519,-0.723060,-4.942924,-4.798901,-4.013011,-2.022188,-11.399353,1.204189,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34904,1.659832,-4.057378,-4.342112,4.883539,6.545916,-7.166761,-20.296010,-4.037425,15.705747,5.379792,...,15.799087,12.155642,-2.779027,-12.929293,-11.685516,-8.569120,-9.950132,-23.678799,-2.239426,2366.79
34905,-0.742430,-1.094278,-3.405028,0.162987,2.476488,-2.828116,-7.237091,-1.078261,6.109831,2.374545,...,6.542620,6.260775,1.004588,-5.206552,-3.366104,-2.250960,-4.466028,-9.016118,-2.830968,2372.10
34906,0.585110,-7.284454,-6.229995,5.868165,5.962808,-4.803534,-30.810923,-1.649579,21.112959,3.222316,...,24.944715,23.846490,5.038551,-21.841349,-11.164002,-5.869021,-14.738364,-28.677563,-3.706940,2600.97
34907,-0.888591,-1.199034,-2.640911,0.001474,3.391269,-1.562954,-9.944487,-1.540009,6.568277,0.901260,...,6.812361,9.019816,2.030966,-8.113600,-2.545458,-1.931313,-7.671851,-10.268171,-3.681764,2793.00


In [10]:
df = df.drop(columns=['Unnamed: 0'])

In [11]:
df

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,y
0,1.021375,-2.071907,0.109130,2.028637,1.647268,-5.600501,-11.082684,-2.937554,8.876988,0.754120,...,10.301312,8.661780,2.696368,-7.467528,-4.141777,-6.063578,-5.465941,-12.236846,-0.982680,0.00
1,-0.473433,-2.432352,-3.370579,-2.080286,2.272056,-2.999003,-9.676762,-3.197955,5.274058,-1.527994,...,8.098873,5.970570,3.107769,-8.935454,-5.169111,-2.756884,-6.406341,-11.645088,-2.429996,0.00
2,0.496656,-3.803691,-2.876035,-2.808888,2.432933,-4.452638,-10.178493,-2.079107,3.339100,0.711161,...,9.255146,5.377076,2.417149,-9.325192,-4.567752,-4.578742,-8.215970,-10.413951,-2.376374,0.00
3,-0.111485,-3.251434,-2.736400,-2.645135,2.578548,-5.707126,-8.472591,-2.248675,4.185829,-0.826409,...,9.187645,5.323506,2.145598,-7.669068,-4.591479,-2.775401,-6.426649,-10.316141,-1.198285,0.00
4,2.230854,-2.023349,-6.237648,5.693612,0.467946,1.374880,-9.434242,-1.290353,2.973960,1.139288,...,9.080440,10.368519,-0.723060,-4.942924,-4.798901,-4.013011,-2.022188,-11.399353,1.204189,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34904,1.659832,-4.057378,-4.342112,4.883539,6.545916,-7.166761,-20.296010,-4.037425,15.705747,5.379792,...,15.799087,12.155642,-2.779027,-12.929293,-11.685516,-8.569120,-9.950132,-23.678799,-2.239426,2366.79
34905,-0.742430,-1.094278,-3.405028,0.162987,2.476488,-2.828116,-7.237091,-1.078261,6.109831,2.374545,...,6.542620,6.260775,1.004588,-5.206552,-3.366104,-2.250960,-4.466028,-9.016118,-2.830968,2372.10
34906,0.585110,-7.284454,-6.229995,5.868165,5.962808,-4.803534,-30.810923,-1.649579,21.112959,3.222316,...,24.944715,23.846490,5.038551,-21.841349,-11.164002,-5.869021,-14.738364,-28.677563,-3.706940,2600.97
34907,-0.888591,-1.199034,-2.640911,0.001474,3.391269,-1.562954,-9.944487,-1.540009,6.568277,0.901260,...,6.812361,9.019816,2.030966,-8.113600,-2.545458,-1.931313,-7.671851,-10.268171,-3.681764,2793.00
